# **Data**

# Covid Worry dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

covidworry_train_csv_path = '/content/drive/My Drive/Colab Notebooks/NLP Augmentation/Datasets/covidworry/covidworry_train.csv'
covidworry_dev_csv_path = '/content/drive/My Drive/Colab Notebooks/NLP Augmentation/Datasets/covidworry/covidworry_test.csv'

covidworry_train = pd.read_csv(covidworry_train_csv_path)
covidworry_dev = pd.read_csv(covidworry_dev_csv_path)


In [ ]:
print('covidworry_train shape:', covidworry_train.shape)
covidworry_train['essay'].head(10)

covidworry_train shape: (1685, 2)


0    I feel stressed because of self isolation and ...
1    At this moment, I am feeling incredibly bored ...
2    Im feeling a combination of anxiety and fear f...
3    Extremely anxious at the Unknown’s in the whol...
4    I feel worried for my friends and family and s...
5    I am worried about family and friends being af...
6    I am scared for my family and friends. I do no...
7    It's a slightly worrying situation especially ...
8    I am quite anxious at the moment as I have a p...
9    Very worried I might get it if go out for supp...
Name: essay, dtype: object

In [ ]:
covidworry_train['emotion'].head(10)

0    3
1    0
2    3
3    3
4    1
5    0
6    0
7    1
8    3
9    4
Name: emotion, dtype: int64

In [ ]:
X_train = covidworry_train['essay']
y_train = covidworry_train['emotion']

X_test = covidworry_dev['essay']
y_test = covidworry_dev['emotion']

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(1685,)
(1685,)
(723,)
(723,)


In [ ]:
# label: 'sadness': 0, 'neutral': 6, 'fear': 2, 'anger': 3, 'disgust': 4, 'surprise': 5, 'joy': 6
y_train.value_counts()

3    966
4    250
1    233
0    161
2     75
Name: emotion, dtype: int64

In [ ]:
y_test.value_counts()

3    415
4    107
1    100
0     69
2     32
Name: emotion, dtype: int64

In [ ]:
from tqdm.notebook import tqdm
tqdm.pandas()

data = {'text': X_train, 'label': y_train}
base_train = pd.DataFrame.from_dict(data)
base_train.shape

(1685, 2)

In [ ]:
base_train.shape

(1685, 2)

# ChatGPT API Augmentation 

In [ ]:
!pip install --upgrade setuptools


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.5 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.
cvxpy 1.2.3 requires setuptools<=64.0.2, but you have setuptools 67.6.0 which is incompatible.


In [ ]:
!pip install openai


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 KB 1.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 10.7 MB/s eta 0:00:00
  Created wheel for openai: filename=openai-0.27.1-py3-none-any.whl size=70091 sha256=0f447150b87b442b2e407a079d9fb01eb48d08f155d1f1e004dbb6b533a33b59
  Stored in directory: /root/.cache/pip/wheels/1f/d1/75/8015df8f7ec8ba5422d8a45786cbb64d4

In [ ]:
rep = 5
chatgpt_train = base_train.copy()

In [ ]:
import openai
import pandas as pd
import time

openai.api_key = "REPLACE WITH SECRET API KEY"

def augment_text(text):

    prompt = (f"Paraphrase this sentence: '{text}'. "
          "Your paraphrased version should be grammatically correct and semantically equivalent.")
    
    response = openai.Completion.create(
        engine="text-davinci-002",
        prompt=prompt,
        max_tokens=60,
        n=5,
        stop=None,
        temperature=0.7,
    )

    responses = []
    for i in range(5):
      responses.append(response.choices[i].text.strip())
    time.sleep(3)
    return responses



In [ ]:
chatgpt_train['paraphrase'] = chatgpt_train['text'].progress_apply(lambda x:augment_text(x))
chatgpt_train = chatgpt_train.explode('paraphrase').reset_index(drop=True)



  0%|          | 0/1685 [00:00<?, ?it/s]

In [ ]:
chatgpt_train.head(50)

,text,label,paraphrase
0,I feel stressed because of self isolation and ...,3,I feel stressed because of self-isolation and ...
1,I feel stressed because of self isolation and ...,3,I feel stressed because of self-isolation and ...
2,I feel stressed because of self isolation and ...,3,I'm feeling stressed because of self-isolation...
3,I feel stressed because of self isolation and ...,3,I feel stressed because of self-isolation and ...
4,I feel stressed because of self isolation and ...,3,I'm feeling stressed out because of self-isola...
5,"At this moment, I am feeling incredibly bored ...",0,I am feeling very bored by the current situati...
6,"At this moment, I am feeling incredibly bored ...",0,"At this moment, I am feeling incredibly bored ..."
7,"At this moment, I am feeling incredibly bored ...",0,I'm feeling really bored right now with the co...
8,"At this moment, I am feeling incredibly bored ...",0,I am currently feeling very bored by the const...
9,"At this moment, I am feeling incredibly bored ...",0,I am feeling incredibly bored and alone due to...


In [ ]:
!cd '/content/drive/My Drive/Colab Notebooks/NLP Augmentation/Datasets/'
chatgpt_train.to_csv('/content/drive/My Drive/Colab Notebooks/NLP Augmentation/Datasets/covidworry_chatgptaug_5.csv', encoding='utf-8', index=False, sep=',')